# Text Analytics Group Assignment #2

**MIS 284N - Section B**   
   
**Members:**   
Patricia Schutter (pls949)   
Sujay Chebbi (sc64344)   
Immanuel Ponminissery (ijp284)   
Vikram Seth (vs22826)   
Lai Jiang (lj9384)

## Reviews of Amazon Headphones Analysis

Headphones are a popular electronic product category with a multitude of different product options and brands. There are several product attributes that consumers tend to heavily consider while making purchase decisions, and the goal of this analysis is to understand the three most important attributes of headphones for consumers, and to recommend three specific products based on those attributes. 

To understand more about the strength of the associations between specific products and these attributes, we scraped 4,286 reviews of 146 unique headphone products from Amazon.

### Task A - Import Packages & Scrape from Amazon

In [ ]:
import requests
import pandas as pd
import nltk
from string import punctuation
import warnings
warnings.filterwarnings('ignore')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import spacy 
import numpy as np
import string
import vaderSentiment

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ImmanuelJP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The following Amazon Scraping code only executes properly in the *Google Collab* environment.

In [ ]:
!pip install selenium
!apt-get -q update # to update ubuntu to correctly run apt install
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import pandas as pd
from pandas import Series, DataFrame 
import re
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-gpu')
chrome_prefs = {}
chrome_options.experimental_options["prefs"] = chrome_prefs
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)

In [ ]:
number_of_pages = 0

def customer(s):
    return s.endswith('#customerReviews') 

def remove_gp(s):
    return not s.startswith('https://www.amazon.com/gp/')

links = []
while number_of_pages<20:
    number_of_pages = number_of_pages+1
    print(number_of_pages)
    address = 'https://www.amazon.com/s?k=headphones&page='
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(600)
    driver.implicitly_wait(600)
    driver.get(address+str(number_of_pages))
    elems = driver.find_elements_by_xpath("//a[@href]")
    for elem in elems:
        links.append(elem.get_attribute("href"))    
        
links_df = DataFrame(links, columns=['Websites'])

#The links_df dataframe contains all the links i.e. links for products and links 
#for Amazon ads and other online content. These links have to be filtered so that only 
#links of products with customer reviews are obtained.

mask = links_df['Websites'].map(customer)
df_2 = links_df[mask].copy()
df_2 = df_2.reset_index(drop = True)

mask_2 = df_2['Websites'].map(remove_gp)
clean_links = df_2[mask_2].copy()
clean_links = clean_links.reset_index(drop = True)

#After obtaining the links of the products that have to be scraped, they are saved to a csv as shown below.
clean_links.to_csv("clean_links_v3.csv")

#Using these links, the links of the reviews for the different products are found and saved in a dataframe.
clean_links = pd.read_csv('clean_links_v3.csv', index_col=[0])

def get_review_link(s):
    review_link = s.split('/dp')[0]+'/product-reviews/'+(s.split('/dp')[1]).lstrip('/').split('/')[0]+'/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=1'
    return review_link

clean_links['review_links'] = clean_links['Websites'].map(get_review_link)

#Using the links for the reviews, the scraper accesses different pages of reviews and collects 
#information regarding a particular review i.e the product being reviewed, the review itself and the rating provided.

data = pd.DataFrame(columns=['name','review','rating'])

for k in clean_links['review_links']:

    page_number = 0
    reviews_per_product = 0
    print(k)
    while reviews_per_product<40 and data.shape[0]<5000:
    page_number = page_number+1
    print(page_number)
    new_address = k+str(page_number)
    name = new_address.lstrip('https://www.amazon.com/').split('/')[0]
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(600)
    driver.get(new_address)

    ids = driver.find_elements_by_xpath("//*[contains(@id,'customer_review-')]")
    if len(ids)==0:
        break


    comment_ids = []
    for i in ids:
        comment_ids.append(i.get_attribute('id'))
    for j in comment_ids:
        rating_element = driver.find_elements_by_xpath('//*[@id="' + j +'"]/div[2]/a[1]')[0]
        rating = rating_element.get_attribute('title')


        review_element = driver.find_elements_by_xpath('//*[@id="' + j +'"]/div[4]/span')[0]
        review = review_element.text

        reviews_per_product = reviews_per_product+1
        print(reviews_per_product)                               
        data.loc[len(data)] = [name,review,rating]

#The data set is then saved and changes are made to it so numerical ratings, on which analysis is 
#conducted later, are obtained.

content = data.drop_duplicates(subset = 'review').copy()
content.to_csv("content.csv")
dataset = pd.read_csv('content.csv', index_col=[0])

def num_rating(s):
    return float(s.split()[0])

dataset['numerical_rating'] = dataset['rating'].map(num_rating)

dataset.to_csv('final_scraped_data.csv')

The above code, when executed, outputs a CSV file titled *final_scraped_data.csv* which we will now read in for further analysis.

In [ ]:
amazon_df = pd.read_csv('final_scraped_data.csv')
amazon_df[:10]

,Unnamed: 0,name,review,rating,numerical_rating
0,0,Sony-MDRZX110-BLK-Stereo-Headphones,We bought these after we broke our light-weigh...,3.0 out of 5 stars,3.0
1,1,Sony-MDRZX110-BLK-Stereo-Headphones,"I used these for a while, and use them as a ba...",3.0 out of 5 stars,3.0
2,2,Sony-MDRZX110-BLK-Stereo-Headphones,When your craft desk is right next to the TV t...,5.0 out of 5 stars,5.0
3,3,Sony-MDRZX110-BLK-Stereo-Headphones,These were purchased during amazons daily deal...,5.0 out of 5 stars,5.0
4,4,Sony-MDRZX110-BLK-Stereo-Headphones,I should of known better then to buy headphone...,3.0 out of 5 stars,3.0
5,5,Sony-MDRZX110-BLK-Stereo-Headphones,These headphones are a little uncomfortable. I...,4.0 out of 5 stars,4.0
6,6,Sony-MDRZX110-BLK-Stereo-Headphones,Bought this originally 8/1/18. Arrived damaged...,1.0 out of 5 stars,1.0
7,7,Sony-MDRZX110-BLK-Stereo-Headphones,Great sound for such a cheap price. Used them ...,2.0 out of 5 stars,2.0
8,8,Sony-MDRZX110-BLK-Stereo-Headphones,"There are subtleties to achieving a soft, comf...",5.0 out of 5 stars,5.0
9,9,Sony-MDRZX110-BLK-Stereo-Headphones,These inexpensive headphones have light-weight...,4.0 out of 5 stars,4.0


### Task B - Attribute Identification

First, we prepared the reviews for each product for a word frequency analysis to understand which attributes are most commonly discussed by customers.

In [ ]:
#REMOVE PUNCTUATION
amazon_df['cleaned_review'] = amazon_df['review'].apply(lambda x :x.translate(str.maketrans('', '', string.punctuation)))

#LOWER CASE EVERY WORD
amazon_df['cleaned_review'] = amazon_df['cleaned_review'].apply(lambda x :x.lower())

#TOKENIZE & TAKE UNIQUE WORDS TO A FREQUENCY LIST
amazon_df['cleaned_review'] = amazon_df['cleaned_review'].apply(word_tokenize).apply(set).apply(list)

def remove_stopwords(s):
    """
    This function removes all stopwords from product reviews in the dataframe.
    """
    list1=[]
    for w in s:
        if w not in stop_words:
            list1.append(w)
    return list1

#CREATE A DATAFRAME COLUMN WITH 'CLEANED' REVIEWS
amazon_df['cleaned_review'] =  amazon_df['cleaned_review'].apply(remove_stopwords)

In [ ]:
#CREATING A COMBINED LIST OF ALL WORDS
all_words = []
for i in range(len(amazon_df)):
    all_words+=amazon_df['cleaned_review'][i]
    
#WORD FREQUENCY
from nltk import FreqDist
word_freq = nltk.FreqDist(all_words)


word_freq.most_common()

[('sound', 2180),
 ('headphones', 1890),
 ('quality', 1581),
 ('great', 1518),
 ('good', 1440),
 ('use', 1171),
 ('like', 977),
 ('comfortable', 972),
 ('price', 959),
 ('would', 879),
 ('noise', 876),
 ('well', 860),
 ('work', 825),
 ('one', 823),
 ('ear', 784),
 ('pair', 765),
 ('really', 759),
 ('get', 725),
 ('also', 687),
 ('bought', 686),
 ('time', 685),
 ('music', 663),
 ('ears', 655),
 ('dont', 642),
 ('’', 614),
 ('even', 578),
 ('im', 566),
 ('better', 565),
 ('much', 561),
 ('fit', 556),
 ('hear', 547),
 ('battery', 545),
 ('phone', 528),
 ('bluetooth', 522),
 ('used', 519),
 ('love', 514),
 ('using', 509),
 ('long', 499),
 ('head', 497),
 ('product', 493),
 ('still', 491),
 ('bass', 482),
 ('volume', 447),
 ('got', 446),
 ('recommend', 446),
 ('buy', 440),
 ('little', 437),
 ('first', 433),
 ('set', 426),
 ('ive', 426),
 ('life', 418),
 ('around', 415),
 ('wear', 398),
 ('nice', 396),
 ('headset', 395),
 ('without', 393),
 ('2', 391),
 ('pretty', 386),
 ('could', 382),
 ('e

From the word frequency analysis, we were able to determine that **three important attributes to consumers** were:

**-Sound   
-Price   
-Comfort**

## Task C - Similarity Analysis

Next, we performed a similarity analysis using *basic cosine similarity* with the three attributes specified above.  
    
We began by defining functions that will call SkLearn cosine_similarity method. 

In [ ]:
#DEFINING FUNCTIONS
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics.pairwise import cosine_similarity

#INPUT LIST FOR COSINE_SIMILARITY
scp = ['sound', 'comfort', 'price']
attributes = " ".join(scp)

def similarity(string):
    documents = [attributes, string]
    tfidf_vectorizer = CountVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)[0][1]

def join_words(comment):   
    """"Concatenates the words in the cleaned reviews"""
    return " ".join(comment)

Next, we apply the above defined functions onto the dataframe in order to obtain the cosine similarity between the reviews and the three attributes specified by the customers. The output contains 3 columns of product name, review, and similarity score.  

In [ ]:
amazon_df['joined_review'] = amazon_df['cleaned_review'].apply(join_words)
amazon_df['cosine_similarity'] = amazon_df['joined_review'].map(similarity)

amazon_df.rename(columns={'name':'product'}, inplace=True)
amazon_df[['product','review','cosine_similarity']]

,product,review,cosine_similarity
0,Sony-MDRZX110-BLK-Stereo-Headphones,We bought these after we broke our light-weigh...,0.166667
1,Sony-MDRZX110-BLK-Stereo-Headphones,"I used these for a while, and use them as a ba...",0.000000
2,Sony-MDRZX110-BLK-Stereo-Headphones,When your craft desk is right next to the TV t...,0.000000
3,Sony-MDRZX110-BLK-Stereo-Headphones,These were purchased during amazons daily deal...,0.146647
4,Sony-MDRZX110-BLK-Stereo-Headphones,I should of known better then to buy headphone...,0.226455
...,...,...,...
4280,Sennheiser-HD-206-Closed-Back-Headphones,My son is very picky about his headphones. He ...,0.204124
4281,Sennheiser-HD-206-Closed-Back-Headphones,Cord just a little too long. Otherwise good va...,0.218218
4282,Sennheiser-HD-206-Closed-Back-Headphones,Great sound. No problems. Comfortable.,0.288675
4283,Sennheiser-HD-206-Closed-Back-Headphones,very good for the price,0.408248


We then create a new dataframe that includes the average cosine similarity between reviews and the three attributes aggregated by product. This dataframe will be used for average sentiment scores and overall average evaluation scores per product later on.

In [ ]:
amazon_average_df = amazon_df.groupby('product')[['cosine_similarity']].mean()
amazon_average_df.head()

,cosine_similarity
product,
AILIHEN-C8-Headphones-Microphone-Smartphones,0.075808
AUKEY-Wireless-Bluetooth-Headphones-Charging,0.086303
AfterShokz-Titanium-Wireless-Conduction-Headphones,0.057942
AmazonBasics-Ear-Headphones-Mic-Black,0.104946
AmazonBasics-Lightweight-On-Ear-Headphones-Black,0.100067


Now that we have the average cosine_similarity between each review and the three attributes aggreated by product, we are able to move forward to seniment analysis for each review and the three features.

## Task D - Sentiment Analysis

In order to understand more context as to how review writers felt about the products when they mentioned the attributes, we performed a sentiment analysis on each review to get a sentiment score realted to each attribute. We started by removing stopwords from all reviews.

In [ ]:
amazon_df['review no stopwords']=amazon_df['review'].apply(lambda x :x.lower()).apply(word_tokenize).apply(remove_stopwords)
amazon_df['review no stopwords']=amazon_df['review no stopwords'].apply(join_words)
amazon_df['review no stopwords']

0       bought broke light-weight sony 's -- - use 'em...
1       used , use backup headphones break . seems lik...
2       craft desk right next tv little brother uses w...
3       purchased amazons daily deals , cost $ 8.65 w/...
4       known better buy headphones price , always lik...
                              ...                        
4280    son picky headphones . really likes sound qual...
4281      cord little long . otherwise good value price .
4282               great sound . problems . comfortable .
4283                                           good price
4284    awesome headphones price : comfy , nice long c...
Name: review no stopwords, Length: 4285, dtype: object

Next, we defined functions that would detect the presence of the key attributes in each review, and if present would return a sentiment score. If the attribute was not mentioned, the score would be left blank as a sentiment score of zero would skew the product sentiment averages.

In [ ]:
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sound_sentiment_analyzer_scores(review):
    if 'sound' in review:
        sentence=re.search(r"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,3}sound(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,3}", review)
        sentence=sentence.group()
        scores = analyser.polarity_scores(sentence)
        scores['compound']
        return scores['compound']

def comfort_sentiment_analyzer_scores(review):
    if 'comfort' in review:
        sentence=re.search(r"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,3}comfort(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,3}", review)
        sentence=sentence.group()
        scores = analyser.polarity_scores(sentence)
        scores['compound']
        return scores['compound']

def price_sentiment_analyzer_scores(review):
    if 'price' in review:
        sentence=re.search(r"(?:[a-zA-Z'-]+[^a-zA-Z'-]+){0,3}price(?:[^a-zA-Z'-]+[a-zA-Z'-]+){0,3}", review)
        sentence=sentence.group()
        scores = analyser.polarity_scores(sentence)
        scores['compound']
        return scores['compound']    

In [ ]:
amazon_df['sound_sentiment']=amazon_df['review no stopwords'].apply(sound_sentiment_analyzer_scores)
amazon_df['comfort_sentiment']=amazon_df['review no stopwords'].apply(comfort_sentiment_analyzer_scores)
amazon_df['price_sentiment']=amazon_df['review no stopwords'].apply(price_sentiment_analyzer_scores)

amazon_df[['product','sound_sentiment','price_sentiment','sound_sentiment']]

,product,sound_sentiment,price_sentiment,sound_sentiment
0,Sony-MDRZX110-BLK-Stereo-Headphones,0.4404,NaN,0.4404
1,Sony-MDRZX110-BLK-Stereo-Headphones,NaN,NaN,NaN
2,Sony-MDRZX110-BLK-Stereo-Headphones,NaN,NaN,NaN
3,Sony-MDRZX110-BLK-Stereo-Headphones,0.2944,0.7003,0.2944
4,Sony-MDRZX110-BLK-Stereo-Headphones,0.6115,0.6908,0.6115
...,...,...,...,...
4280,Sennheiser-HD-206-Closed-Back-Headphones,0.4754,NaN,0.4754
4281,Sennheiser-HD-206-Closed-Back-Headphones,NaN,0.6486,NaN
4282,Sennheiser-HD-206-Closed-Back-Headphones,0.6908,NaN,0.6908
4283,Sennheiser-HD-206-Closed-Back-Headphones,NaN,0.4404,NaN


After obtaining a sentiment score for each review for each of the three attributes, we then aggregate the average sentiment per attribute per product. After that, we create an overall average sentiment score for the product based on the total average sentiment scores that it held for each of the attributes.  

In [ ]:
amazon_average_df = amazon_df.groupby('product')[['price_sentiment','comfort_sentiment','sound_sentiment',
                                                  'numerical_rating','cosine_similarity']].mean()


amazon_average_df['average_sentiment'] = (amazon_average_df['price_sentiment'] + amazon_average_df['comfort_sentiment']
                                          +amazon_average_df['sound_sentiment'])/3


amazon_average_df.head()

,price_sentiment,comfort_sentiment,sound_sentiment,numerical_rating,cosine_similarity,average_sentiment
product,,,,,,
AILIHEN-C8-Headphones-Microphone-Smartphones,0.179200,0.494708,0.153900,3.233333,0.075808,0.275936
AUKEY-Wireless-Bluetooth-Headphones-Charging,0.284507,0.346210,0.489157,4.133333,0.086303,0.373291
AfterShokz-Titanium-Wireless-Conduction-Headphones,0.254050,0.288309,0.083662,4.166667,0.057942,0.208674
AmazonBasics-Ear-Headphones-Mic-Black,0.444811,0.361200,0.434050,4.200000,0.104946,0.413354
AmazonBasics-Lightweight-On-Ear-Headphones-Black,0.362763,0.521367,0.337493,3.000000,0.100067,0.407208


## Task E - Evaluation Score

We then calculated the average evalution score per product, which is a sum of the average cosine similarity score and average feature sentiment score.   
   
We then reccomend to the customer the three products with the highest evaluation scores.

In [ ]:
amazon_average_df['evaluation_score'] = amazon_average_df['average_sentiment'] + amazon_average_df['cosine_similarity']
amazon_average_df['evaluation_score'].sort_values(ascending=False)[:3]

product
Bluetooth-Headphones-AMZANE-Wireless-Earphones          0.783554
Headphones-gorsun-Lightweight-Adjustable-Smartphones    0.760635
Headphones-i41-Childrem-Adjustable-Cellphones           0.723459
Name: evaluation_score, dtype: float64

For consumers who value Sound, Price, and Quality attributes in headphone products the most, we recommend the [Gorsun Kids Headphones](https://www.amazon.com/Headphones-gorsun-Lightweight-Adjustable-Smartphones/dp/B019Q03128/), the [AMZANE Bluetooth Earbuds](https://www.amazon.com/Bluetooth-Headphones-AMZANE-Wireless-Earphones/dp/B08CKQVP47/), or the [Elecder i41 Kids Headphones](https://www.amazon.com/Headphones-i41-Childrem-Adjustable-Cellphones/dp/B07G75SH2G/).|

## Task F - Spacy Similarity

In order to compare our recommendations via basic cosine similarity and Spacy word vector similarity, we first must import the en_core_web_lg from spacy and define and execute a function to calculate the spacy similarity scores for each review. 

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')
 

def spacy_similarity(comment):
    """Compute the spacy cosine similarity score"""
    base = nlp(comment)
    compare = nlp(attributes)
    return base.similarity(compare)

In [ ]:
amazon_df['spacy_similarity'] = amazon_df['joined_review'].apply(spacy_similarity)
cosine_df = amazon_df[['product','review','spacy_similarity']]
cosine_df.head()

,product,review,spacy_similarity
0,Sony-MDRZX110-BLK-Stereo-Headphones,We bought these after we broke our light-weigh...,0.750864
1,Sony-MDRZX110-BLK-Stereo-Headphones,"I used these for a while, and use them as a ba...",0.674121
2,Sony-MDRZX110-BLK-Stereo-Headphones,When your craft desk is right next to the TV t...,0.652901
3,Sony-MDRZX110-BLK-Stereo-Headphones,These were purchased during amazons daily deal...,0.711295
4,Sony-MDRZX110-BLK-Stereo-Headphones,I should of known better then to buy headphone...,0.738959


After finding the spacy similarity score for each individual review, we found the mean score per product and merged it into the dataframe holding the other average scores per product.

In [ ]:
product_cosine_df = cosine_df.groupby('product')[['spacy_similarity']].mean()
merged_df = amazon_average_df.merge(product_cosine_df, left_on = 'product', right_on = 'product')
merged_df.head()

,price_sentiment,comfort_sentiment,sound_sentiment,numerical_rating,cosine_similarity,average_sentiment,evaluation_score,spacy_similarity
product,,,,,,,,
AILIHEN-C8-Headphones-Microphone-Smartphones,0.179200,0.494708,0.153900,3.233333,0.075808,0.275936,0.351744,0.678642
AUKEY-Wireless-Bluetooth-Headphones-Charging,0.284507,0.346210,0.489157,4.133333,0.086303,0.373291,0.459594,0.696304
AfterShokz-Titanium-Wireless-Conduction-Headphones,0.254050,0.288309,0.083662,4.166667,0.057942,0.208674,0.266616,0.690696
AmazonBasics-Ear-Headphones-Mic-Black,0.444811,0.361200,0.434050,4.200000,0.104946,0.413354,0.518300,0.676802
AmazonBasics-Lightweight-On-Ear-Headphones-Black,0.362763,0.521367,0.337493,3.000000,0.100067,0.407208,0.507274,0.653017


We then were able to ascertain the entire composite evaluation score for each product, using the same average senitment values but substituting cosine_similarity with word vector similarity.

In [ ]:
merged_df['evaluation_new'] = merged_df['average_sentiment'] + merged_df['spacy_similarity']
merged_df['evaluation_new'].sort_values(ascending = False)[:3]

product
Bluetooth-Headphones-AMZANE-Wireless-Earphones             1.345060
Headphones-gorsun-Lightweight-Adjustable-Smartphones       1.282773
Panasonic-Headphones-RP-HF100M-K-Integrated-Travel-Fold    1.278770
Name: evaluation_new, dtype: float64

When utilizing a word vector instead of plain vanilla bag-of-words cosine similarity, the three products with the overall score for recommendation to consumers who value Sound, Price, and Quality attributes in headphone products the most, we recommend the [Gorsun Kids Headphones](https://www.amazon.com/Headphones-gorsun-Lightweight-Adjustable-Smartphones/dp/B019Q03128/), the [AMZANE Bluetooth Earbuds](https://www.amazon.com/Bluetooth-Headphones-AMZANE-Wireless-Earphones/dp/B08CKQVP47/), or the [Panasonic Integrated Travel-Fold Headphones](https://www.amazon.com/Panasonic-Headphones-RP-HF100M-K-Integrated-Travel-Fold/dp/B01HY6WQS6/).

After utilizing Spacy similarity as opposed to the bag-of-words approach, we notice that while 2 of the 3 recommendations are the same, they are in a different order. These products' average basic cosine similarities are all relatively low compared to their average spacy similarities.
   
To investigate this further, we determined the percentage of reviews of the top recommendation for each method that actually mention one of the preferred attributes.

In [ ]:
def percentage_of_attributes(review):
    if 'sound' in review.lower():
        return 1
    elif 'comfort' in review.lower():
        return 1
    elif 'price' in review.lower():
        return 1
    else:
        return 0
    
amazon_df['presence_attribute'] = amazon_df['review'].map(percentage_of_attributes)

In [ ]:
percentage_df = amazon_df.groupby('product')['presence_attribute'].agg(['sum', 'count'])
percentage_df.columns = ['reviews mentioning any attribute','total reviews']
percentage_df['percentage_attributes'] = percentage_df['reviews mentioning any attribute'] / percentage_df['total reviews']

In [ ]:
recommend_df = merged_df.merge(percentage_df, left_on = 'product', right_on = 'product')
recommend_df.loc[['Panasonic-Headphones-RP-HF100M-K-Integrated-Travel-Fold', 
                   'Headphones-i41-Childrem-Adjustable-Cellphones',
                   'Headphones-gorsun-Lightweight-Adjustable-Smartphones',
                  'Bluetooth-Headphones-AMZANE-Wireless-Earphones']]

,price_sentiment,comfort_sentiment,sound_sentiment,numerical_rating,cosine_similarity,average_sentiment,evaluation_score,spacy_similarity,evaluation_new,reviews mentioning any attribute,total reviews,percentage_attributes
product,,,,,,,,,,,,
Panasonic-Headphones-RP-HF100M-K-Integrated-Travel-Fold,0.832360,0.526010,0.537473,3.933333,0.083999,0.631948,0.715946,0.646823,1.278770,16,30,0.533333
Headphones-i41-Childrem-Adjustable-Cellphones,0.644040,0.557522,0.647493,4.933333,0.107108,0.616352,0.723459,0.658182,1.274534,21,30,0.700000
Headphones-gorsun-Lightweight-Adjustable-Smartphones,0.675450,0.572600,0.819386,3.833333,0.071489,0.689145,0.760635,0.593628,1.282773,11,30,0.366667
Bluetooth-Headphones-AMZANE-Wireless-Earphones,0.623843,0.533175,0.674767,5.000000,0.172959,0.610595,0.783554,0.734465,1.345060,26,30,0.866667


When comparing the impact of using basic cosine similarity or spacy similarity as a component of our overall evaluation score, we must compare aspects of the two products that substitute each other. When we switch from using basic cosine similarity to spacy, the only recommended product that is substituted out is the *Elecder i41 Kids Headphones*. 



the attributes look towardsattributes such total percentage of reviews that included at least one of our key attributes

## Task G - Highest Simple Rating

In [ ]:
mask = merged_df['numerical_rating'] == 5.0
merged_df[mask]

,price_sentiment,comfort_sentiment,sound_sentiment,numerical_rating,cosine_similarity,average_sentiment,evaluation_score,spacy_similarity,evaluation_new
product,,,,,,,,,
BiFee-Wireless-Bluetooth-Headphones-Waterproof,0.700300,NaN,0.571900,5.0,0.308607,NaN,NaN,0.756954,NaN
Bluetooth-Headphones-AMZANE-Wireless-Earphones,0.623843,0.533175,0.674767,5.0,0.172959,0.610595,0.783554,0.734465,1.345060
Bluetooth-Headphones-True-Waterproof-Microphone-Workout-Black,0.517150,0.468122,0.621589,5.0,0.123502,0.535620,0.659122,0.722044,1.257664
Cancelling-Headphones-AIKELA-Bluetooth-Headphone,0.337438,0.637750,0.592494,5.0,0.100536,0.522560,0.623096,0.737648,1.260209
EKSA-Ultralight-USB-Gaming-Headset,0.575350,0.587557,0.543010,5.0,0.128340,0.568639,0.696979,0.693994,1.262633


Fom the list of 5 devices that have a 'numerical_rating' of 5.0, the one that has also been recommended in our analyses in Tasks F and E is the 'Bluetooth-Headphones-AMZANE-Wireless-Earphones'. 

It is clear that the devices mentioned above would not meet the attribute requirements of a recommender system for the following reasons:

   
a)The 'BiFee-Wireless-Bluetooth-Headphones-Waterproof' does not have a value for the 'comfort' sentiment. This is indicative of the fact that this device does not cater to the needs of individuals who value comfort as it was never mentioned in a review.
   
b)The 'Cancelling-Headphones-AIKELA-Bluetooth-Headphone' has a relatively low score for the 'price' sentiment and in this context, it is critical that every product has a substantially high score  for all the  desired attributes.
   
c) While the remaining products that were not recommended do not have substantially low 'evaluation_score', the products that were recommended in Tasks E and F do surpass them in said score. The purpose of the recommender system is to offer products that resonate with the users' specific attributes. 

